# Fickling

https://github.com/trailofbits/fickling/tree/master

More examples to explore:

https://github.com/trailofbits/fickling/tree/master/example

In [ ]:
!pip install fickling

In [14]:
import os
import pickle
import tempfile
from pprint import pprint

import fickling

fickling.always_check_safety()

In [54]:
obj = "Hello world"
pickled_obj = pickle.dumps(obj)

try:
    pickle.load(pickled_obj)
except fickling.exception.UnsafeFileError as e:
    pprint(e.info)

In [ ]:
class SafeClass:
    def __init__(self, value):
        self.value = value

    def __reduce__(self):
        return (self.__class__, (self.value,))

obj = SafeClass("Hello world")
pickled_obj = pickle.dumps(obj)

try:
    pickle.load(pickled_obj)
except fickling.exception.UnsafeFileError as e:
    pprint(e.info)

{'analysis': '`from __main__ import SafeClass` imports a Python module that is '
             'not a part of the standard library; this can execute arbitrary '
             'code and is inherently unsafe\n'
             "Call to `SafeClass('Hello world')` can execute arbitrary code "
             'and is inherently unsafe\n'
             "Variable `_var0` is assigned value `SafeClass('Hello world')` "
             'but unused afterward; this is suspicious and indicative of a '
             'malicious pickle file',
 'detailed_results': {'AnalysisResult': {'NonStandardImports': 'from __main__ '
                                                               'import '
                                                               'SafeClass',
                                         'OvertlyBadEval': "SafeClass('Hello "
                                                           "world')",
                                         'UnusedVariables': ('_var0',
                                

In [59]:
class UnsafeClass:
    def __init__(self):
        self.cmd = "echo 'This could be a malicious script'"

    def __reduce__(self):
        return (os.system, (self.cmd,))

obj = UnsafeClass()
pickled_obj = pickle.dumps(obj)

try:
    pickle.load(pickled_obj)
except fickling.exception.UnsafeFileError as e:
    pprint(e.info)

{'analysis': '`from posix import system` is suspicious and indicative of an '
             'overtly malicious pickle file\n'
             'Variable `_var0` is assigned value `system(...)` but unused '
             'afterward; this is suspicious and indicative of a malicious '
             'pickle file',
 'detailed_results': {'AnalysisResult': {'UnsafeImports': 'from posix import '
                                                          'system',
                                         'UnusedVariables': ('_var0',
                                                             'system(...)')}},
 'severity': 'LIKELY_OVERTLY_MALICIOUS'}


In [60]:
with tempfile.NamedTemporaryFile(delete=False) as temp_file:
    pickle.dump(obj, temp_file)

In [61]:
os.system(f"fickling --check-safety {temp_file.name}")
# 0 = safe
# 256, 512 = unsafe

256

In [ ]:
os.system(f"fickling --trace {temp_file.name}")
# Pickle virtual machine opcodes:
# https://github.com/Legoclones/pickledoc/blob/main/Opcodes.md

PROTO
FRAME
SHORT_BINUNICODE
	Pushed 'posix'
MEMOIZE
	Memoized 0 -> 'posix'
SHORT_BINUNICODE
	Pushed 'system'
MEMOIZE
	Memoized 1 -> 'system'
STACK_GLOBAL
	from posix import system
	Popped 'system'
	Popped 'posix'
	Pushed system
MEMOIZE
	Memoized 2 -> system
SHORT_BINUNICODE
	Pushed "echo 'This could be a malicious script'"
MEMOIZE
	Memoized 3 -> "echo 'This could be a malicious script'"
TUPLE1
	Popped "echo 'This could be a malicious script'"
	Pushed ("echo 'This could be a malicious script'",)
MEMOIZE
	Memoized 4 -> ("echo 'This could be a malicious script'",)
REDUCE
	_var0 = system("echo 'This could be a malicious script'")
	Popped ("echo 'This could be a malicious script'",)
	Popped system
	Pushed _var0
MEMOIZE
	Memoized 5 -> _var0
STOP
	result0 = _var0
	Popped _var0
from posix import system
_var0 = system("echo 'This could be a malicious script'")
result0 = _var0


0